In [1]:
# **1: Data Loading And Processing**

import os
import time

import cv2
import numpy as np

import tensorflow as tf
import tensorflow.keras.backend as K

# Define the path to the folder containing the spectrogram images
data_dir = r'path to dataset folder'

# Define the classes (folder names)
classes = os.listdir(data_dir)
num_classes = len(classes)

# Define the desired width and height for resizing
desired_width = 227
desired_height = 227

# Initialize lists to store spectrogram images and their corresponding labels
images_list = []
labels_list = []

# Iterate through each class folder
for class_idx, class_name in enumerate(classes):
    class_dir = os.path.join(data_dir, class_name)
    # Iterate through each spectrogram image in the class folder
    for img_name in os.listdir(class_dir):
        img_path = os.path.join(class_dir, img_name)
        # Read the image using OpenCV
        img = cv2.imread(img_path)
        # Resize the image to the desired dimensions without preserving the aspect ratio
        img = cv2.resize(img, (desired_width, desired_height))
        # Convert the image to float32 and normalize pixel values to [0, 1]
        img = img.astype(np.float32) / 255.0
        # Append the preprocessed image and its label to the lists
        images_list.append(img)
        labels_list.append(class_idx)  # Use class index as label

# Convert the lists to numpy arrays
images = np.array(images_list)
labels = np.array(labels_list)

# Convert the lists to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Shuffle the data
indices = np.arange(images.shape[0])
np.random.shuffle(indices)
images = images[indices]
labels = labels[indices]

# Split the data into training, validation, and test sets (e.g., 80-10-10 split)
train_split = int(0.8 * len(images))
val_split = int(0.10 * len(images))
x_train, y_train = images[:train_split], labels[:train_split]
x_val, y_val = images[train_split:train_split + val_split], labels[train_split:train_split + val_split]
x_test, y_test = images[train_split + val_split:], labels[train_split + val_split:]

In [ ]:
import keras
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
# from keras.layers.advanced_activations import LeakyReLU
import tensorflow as tf
from tensorflow import keras
import keras.layers as layers

model = keras.Sequential()
model.add(layers.Conv2D(filters=96, kernel_size=(11, 11), 
                        strides=(4, 4), activation="relu", 
                        input_shape=(227, 227, 3)))
model.add(layers.BatchNormalization())
model.add(layers.MaxPool2D(pool_size=(3, 3), strides= (2, 2)))
model.add(layers.Conv2D(filters=256, kernel_size=(5, 5), 
                        strides=(1, 1), activation="relu", 
                        padding="same"))
model.add(layers.BatchNormalization())
model.add(layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2)))
model.add(layers.Conv2D(filters=384, kernel_size=(3, 3), 
                        strides=(1, 1), activation="relu", 
                        padding="same"))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(filters=384, kernel_size=(3, 3), 
                        strides=(1, 1), activation="relu", 
                        padding="same"))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(filters=256, kernel_size=(3, 3), 
                        strides=(1, 1), activation="relu", 
                        padding="same"))
model.add(layers.BatchNormalization())
model.add(layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(4096, activation="relu"))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(17, activation="softmax"))
model.compile(loss='sparse_categorical_crossentropy', 
              optimizer=tf.optimizers.SGD(learning_rate=0.001), 
              metrics=['accuracy'])
model.summary()

In [ ]:
"""# **3:Training**"""

# Define number of epochs and batch size
epochs = 1
batch_size = 32

# Train the model
history =model.fit(
    x_train,  # Pass the left and right images as inputs
    y_train,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(x_val, y_val),  # Pass validation data
    verbose=1
)

In [ ]:
y_pred = model.predict(x_val)
y_pred_classes = y_pred.argmax(axis=1)  # Convert probabilities to class indices

from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

# Calculate precision, recall, F1-score
precision = precision_score(y_val, y_pred_classes, average='macro')  # Use 'weighted' if class imbalance exists
recall = recall_score(y_val, y_pred_classes, average='macro')
f1 = f1_score(y_val, y_pred_classes, average='macro')

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


In [ ]:
"""# **4:Evaluation**"""

from sklearn.metrics import (
    accuracy_score,
    f1_score,
    fbeta_score,
    precision_score,
    recall_score,
)

# Use the trained model to predict classes for the test set
y_pred =model.predict(x_test,)
y_pred_classes = np.argmax(y_pred, axis=1)

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred_classes)
precision = precision_score(y_test, y_pred_classes, average='weighted')
recall = recall_score(y_test, y_pred_classes, average='weighted')
f1 = f1_score(y_test, y_pred_classes, average='weighted')
f2 = fbeta_score(y_test, y_pred_classes, average='weighted', beta=2)

print("Accuracy: %.5f", accuracy)
print("Precision: %.5f", precision)
print("Recall: %.5f", recall)
print("F1-score: %.5f", f1)
print("F2-score: %.5f", f2)

In [ ]:
"""# **5:Confusion Matrix**"""

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Generate confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred_classes)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Assuming y_test and y_pred_classes are defined
# Generate confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred_classes)

# Generate normalized confusion matrix
conf_matrix_normalized = confusion_matrix(y_test, y_pred_classes, normalize='true')

# Set up subplots to display both matrices side by side
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Plot normal confusion matrix
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes, ax=axes[0])
axes[0].set_title('Confusion Matrix (Counts)')
axes[0].set_xlabel('Predicted Labels')
axes[0].set_ylabel('True Labels')

# Plot normalized confusion matrix
sns.heatmap(conf_matrix_normalized, annot=True, fmt='.2f', cmap='Blues', xticklabels=classes, yticklabels=classes, ax=axes[1])
axes[1].set_title('Normalized Confusion Matrix (Proportions)')
axes[1].set_xlabel('Predicted Labels')
axes[1].set_ylabel('True Labels')

# Adjust layout and display the plot
plt.tight_layout()
plt.savefig("conf.png", dpi=650)
plt.show()


In [ ]:
"""# **8:PREDICTION**"""

import cv2
import numpy as np

# Load the external spectrogram
external_spectrogram_path = '/content/siamese_ecg_prod/dataset/DatasetWave_2/4/577.png'
external_spectrogram = cv2.imread(external_spectrogram_path)

# Preprocess the external spectrogram
desired_width = 224
desired_height = 224
external_spectrogram_resized = cv2.resize(external_spectrogram, (desired_width, desired_height))
external_spectrogram_normalized = external_spectrogram_resized.astype(np.float32) / 255.0

# Reshape the input to match model's input shape (add batch dimension)
input_external = np.expand_dims(external_spectrogram_normalized, axis=0)

# Use the trained model to predict classes for the external spectrogram
predictions = siamese_model.predict([input_external, input_external])

# Convert predictions to class labels
predicted_class_index = np.argmax(predictions)

# Print the predicted class label
print("Predicted class:", classes[predicted_class_index])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize

# Assuming y_test and y_pred_proba are defined, and num_classes is the number of classes
# Binarize the output for multi-class ROC
y_pred_proba = siamese_model.predict([x_test, x_test])
y_test_bin = label_binarize(y_test, classes=np.arange(num_classes))

# Compute micro-average ROC curve and AUC
fpr_micro, tpr_micro, _ = roc_curve(y_test_bin.ravel(), y_pred_proba.ravel())
roc_auc_micro = auc(fpr_micro, tpr_micro)

# Plot micro-average ROC curve
plt.figure(figsize=(10, 8))
plt.plot(fpr_micro, tpr_micro, color='blue', lw=2,
         label='Micro-average ROC curve (AUC = %0.2f)' % roc_auc_micro)

# Plot diagonal line for random guessing
plt.plot([0, 1], [0, 1], 'k--')

# Set limits and labels
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Micro-average Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.savefig('roc.png', dpi= 650)
plt.show()

# Print overall micro-average AUC
# print(f'Micro-average AUC: {roc_auc_micro:.2f}')


In [ ]:
"""# **10: Accuracy Curve**"""

import matplotlib.pyplot as plt

# Extract training and validation accuracy from history
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

# Plot accuracy curve
plt.figure(figsize=(10, 6))
plt.plot(range(1, epochs + 1), train_accuracy, label='Training Accuracy')
plt.plot(range(1, epochs + 1), val_accuracy, label='Validation Accuracy')
plt.title('Accuracy Curve')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig('acc.png', dpi=650)
plt.show()